In [10]:
df_1 = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/5years_kospi.csv', index_col=0)
df_2 = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/5years_kosdaq.csv', index_col=0)
df_raw = pd.concat([df_1,df_2],ignore_index=True)

#### Making Market Timming

In [3]:
import pandas as pd

In [1]:
def making_mt(start,end,code):
    from pykrx import stock
    df = stock.get_index_fundamental(start,end,code)
    df = df.drop(['등락률','PER','선행PER','PBR','배당수익률'],axis=1)
    
    ''' code : KOSPI ('1001') KOSDAQ ('2001') '''
    if code=='1001': 
        df['sma20'] = df['종가'].rolling(20).mean()
        df['mt'] = df['종가']>df['sma20']
        df = df.dropna()
        return df
    elif code=='2001':     
        df['sma3'] = df['종가'].rolling(3).mean()
        df['sma5'] = df['종가'].rolling(5).mean()
        df['sma10'] = df['종가'].rolling(10).mean()
        df['mt'] = ((df['종가']>df['sma3'])|(df['종가']>df['sma5'])|(df['종가']>df['sma10']))
        df = df.dropna()
        return df
    else:
        print('올바른 종목 code 입력')

In [5]:
def making_mt_plot(df,code):
    import matplotlib.pyplot as plt
    from matplotlib import dates
    if code=='1001': 
        plt.figure(figsize = (50, 10))
        plt.title("Kospi Market timming")
        plt.plot(df["종가"])
        plt.plot(df['sma20'])
        plt.scatter(df.loc[df['mt']==1].index, df['종가'][df['mt']==1],
                    color = 'red',
                    label='Go',
                    marker ='^',
                    alpha=0.8)
        plt.scatter(df.loc[df['mt']==0].index, df['종가'][df['mt']==0],
                    color = 'blue',
                    label='Stop',
                    marker ='v',
                    alpha=0.8)
        plt.xticks(rotation=45)
        plt.legend()
        plt.show()
    elif code=='2001':
        plt.figure(figsize = (50, 10))
        plt.title("Kosdaq Market timming")
        plt.plot(df["종가"])
        plt.plot(df['sma3'])
        plt.plot(df['sma5'])
        plt.plot(df['sma10'])
        plt.scatter(df.loc[df['mt']==1].index, df['종가'][df['mt']==1],
                    color = 'red',
                    label='Go',
                    marker ='^',
                    alpha=0.8)
        plt.scatter(df.loc[df['mt']==0].index, df['종가'][df['mt']==0],
                    color = 'blue',
                    label='Stop',
                    marker ='v',
                    alpha=0.8)
        plt.xticks(rotation=45)
        plt.legend()
        plt.show()
    else:
        print('Error : Code check')
    

## 설명 
* backtest 시작 , 끝날짜를 for 문으로 돌린다 ex) for i in range(start,end)  
* 시작 날짜에 maket Timming 이 true면 아래전략 시행  
* sort_vol = 시작 날짜 누적거래대금 범위에 속하는 종목 df로만든다.  output : df_vol
* search momentum signal = df_vol과 df_raw 를 활용하여 Momentum 값을 계산하고 signal을 만든다 
* True 인 종목을 매수한다. 현금계좌잔고 감소 , 보유잔고 증가 (주가*수량)  

<익일>  
* 다음날 보유잔고에 매도 조건을 확인한다 매도조건이 True이면 매도한다. 현금계좌잔고 증가 (당일 종가 * 수량) 보유잔고 감소  
* 매도 후 전일과 같이 위 과정을 반복  

<최종>  
* 수익률 출력  
* MDD 출력 (이전략으로 매매를 했을때 최대 몇% 까지 손실을 보았었는가)  
* 샤프지수출력 (이 전략이아니라 그냥 지수에 투자했을 때에 비해 변동성이 얼마나 큰가 )
* 승률 (이전략을 했을때 + 수익을 본횟수 / 전체횟수)  
* log (활용 무궁무진 : 매수 매도 기록을 전부기록 추후 확률 통계에 기반이됨)  

<고급>
* momentum 지수가 몇일 때 매수를 하면 다음날 상승확률이 가장 높은지 통계적검증 해서 파라미터 조정가능 


#### Making 거래대금조회


In [17]:
def sort_vol(df,date,range):
    df = df[(df['누적거래대금']>=range)&(df['날짜']==date)]
    return df

#### Momentum Search

In [10]:
def search_momentum_signal(df_vol, df_raw):
    li = df_vol['종목명'].values
    dt = df_vol['날짜'].values[0]
    df_vol['Buy'] = None
    df_vol['momentum'] = None
    for i in li:
        today_idx = df_raw[(df_raw['종목명']==i)&(df_raw['날짜']==dt)].index
        day5_idx = df_raw[(df_raw['종목명']==i)&(df_raw['날짜']==dt)].index-5
        momentum = df_raw.loc[today_idx]['종가'].values/df_raw.loc[day5_idx]['종가'].values
        df_vol.loc[df_vol['종목명']==i,'momentum']=momentum[0]
        df_vol.loc[df_vol['종목명']==i,'Buy']=df_vol['momentum']>=1
    return df_vol
        

In [16]:
df_vol = sort_vol(df_raw,20210104,10000000000)
search_momentum_signal(df_vol, df_raw)

C:\Users\mi-bi\AppData\Local\Temp\ipykernel_24240\838232746.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vol['Buy'] = None
C:\Users\mi-bi\AppData\Local\Temp\ipykernel_24240\838232746.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vol['momentum'] = None


,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,Buy,momentum
751,20210104,62200,59700,63600,59000,486476,29866340900,LX세미콘,True,1.132969
7118,20210104,37050,37900,37900,36600,431008,15956584950,PI첨단소재,True,1.027739
9039,20210104,41200,40950,41600,40600,795524,32678001600,엠씨넥스,True,1.036478
13464,20210104,5164,4997,5194,4987,2217332,11171478410,ESR켄달스퀘어리츠,True,1.011557
13976,20210104,48400,47150,48600,45650,4184952,198363480450,명신산업,True,1.059081
...,...,...,...,...,...,...,...,...,...,...
1603574,20210104,2510,2375,2520,2285,7625539,18510920845,모베이스전자,True,1.07957
1606002,20210104,2900,2740,2915,2700,5157323,15446749035,국영지앤엠,True,1.104762
1612072,20210104,7930,8200,8200,7710,1313680,10395376730,태광,True,1.155977
1624212,20210104,9180,9070,9300,8900,1284486,11783642580,유니크,True,1.103365


#### 모멘텀 문제 있어 두가지전략 가지고가야함 

In [13]:
# raw 파일에 전종목 momentum 을 만들고 시작하는방법
def make_momentum(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        df.loc[idx[0]:idx[-1],'momentum'] = df.loc[idx[0]:idx[-1],'종가']/df.loc[idx[0]:idx[-1],'종가'].shift(5)*100
    return df

In [14]:
momentum_df = make_momentum(df_raw)

In [18]:
df_vol = sort_vol(momentum_df,20210104,10000000000)

In [24]:
con1 = (momentum_df['momentum']>=100)&(momentum_df['momentum'].shift(1)<100)

In [28]:
test = momentum_df[momentum_df['종목명']=='삼성전자']

In [39]:
test = test.dropna()

In [47]:
test[(test['momentum']>=100)&(test['momentum'].shift(1)<100)]

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,momentum,signal
877215,20180213,47540,46200,48060,46200,18923250,896530740450,삼성전자,100.253058,True
877228,20180307,48620,48200,48900,47220,21427050,1036655011000,삼성전자,102.617138,True
877239,20180322,51780,51060,51780,51040,8454100,434965794173,삼성전자,100.465658,True
877251,20180409,49200,48260,49440,48200,9950400,488395543080,삼성전자,101.359703,True
877265,20180427,53000,53380,53640,52440,30310800,1611240055340,삼성전자,102.673382,True
...,...,...,...,...,...,...,...,...,...,...
878394,20221129,60600,59900,60600,59800,7014160,422226001673,삼성전자,100.000000,True
878398,20221205,60300,60900,61100,60000,13767787,832613739000,삼성전자,100.332779,True
878402,20221209,60400,59300,60600,59300,10529632,633777717900,삼성전자,100.000000,True
878404,20221213,59700,59600,60300,59400,9040873,539587782200,삼성전자,100.844595,True


In [48]:
test.loc[877227]

날짜              20180306
종가                 47020
시가                 45920
최고가                47100
최저가                45820
누적거래량           15423350
누적거래대금      719307895174
종목명                 삼성전자
momentum       99.240186
signal             False
Name: 877227, dtype: object

In [25]:
for i in momentum_df['종목명'].unique:
    momentum_df.loc[momentum_df['종목명']==i]

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,momentum
12,20221206,8340,6510,8340,6450,7523023,56764551840,유니드비티플러스,120.869565
24,20221222,8150,7060,8700,7060,8367002,68443673110,유니드비티플러스,100.741656
40,20180212,53200,54100,54200,52300,91572,4892513000,LX세미콘,102.504817
42,20180214,52900,52900,53500,51600,72788,3812474500,LX세미콘,100.000000
57,20180312,43550,42100,43550,42100,171087,7364499500,LX세미콘,105.447942
...,...,...,...,...,...,...,...,...,...
2621608,20221124,4005,4070,4070,3980,27998,112240160,모헨즈,101.264223
2621611,20221129,4060,3810,4070,3810,36164,142443995,모헨즈,103.969270
2621621,20221213,4095,3910,4180,3910,79585,324832050,모헨즈,100.244798
2621625,20221219,3965,3905,4015,3905,19806,78433040,모헨즈,101.406650


In [49]:
df_vol = sort_vol(momentum_df,20210104,10000000000)

In [67]:
for i in momentum_df['종목명'].unique():
    momentum_df['signal'] = momentum_df['날짜']==x 

날짜            20201230
종가               13450
시가               13650
최고가              13650
최저가              13400
누적거래량            97405
누적거래대금      1314306400
종목명             하이록코리아
momentum     97.818182
Name: 2616287, dtype: object

In [55]:
# vol 날짜에 vol 종목명에 momentum 전날 momentum 값 

In [76]:
momentum_df

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,momentum
0,20221222,10650,9980,11700,9930,28856232,309808795730,바이오노트,NaN
1,20221223,10150,10600,11200,10100,8180753,86595242100,바이오노트,NaN
2,20221226,9720,10550,10700,9650,4479667,45206813740,바이오노트,NaN
3,20221227,9360,9770,9880,9130,4101930,38365054640,바이오노트,NaN
4,20221228,8870,9390,9420,8700,3177379,28837782380,바이오노트,NaN
...,...,...,...,...,...,...,...,...,...
2621629,20221223,3765,3900,3900,3765,33477,127543190,모헨즈,95.195954
2621630,20221226,3720,3765,3765,3610,30218,111265525,모헨즈,93.820933
2621631,20221227,3755,3700,3755,3665,17856,66261900,모헨즈,100.133333
2621632,20221228,3630,3770,3775,3615,25279,93477675,모헨즈,95.275591
